# 🤖 SMS Spam Classification — Deep Learning & Bayesian Optimization

This notebook constitutes the second phase of our project: **Advanced Modeling and Hyperparameter Optimization**. Having completed a thorough **Exploratory Data Analysis (EDA)** in the previous stage, we now leverage those insights to construct a sophisticated deep learning architecture. 

Our objective is to move beyond heuristic-based classification and implement a **Bidirectional Long Short-Term Memory (Bi-LSTM)** network. This model is engineered to interpret the complex, sequential relationships within SMS text, allowing for a more nuanced distinction between legitimate communication (Ham) and unsolicited outreach (Spam).

---

## 🔍 Bridging the Gap: Insights from EDA
The architectural choices in this notebook are not arbitrary; they are the direct result of our findings in the EDA phase:
* **The Imbalance Challenge**: We confirmed that **86.6%** of our data is "Ham." To ensure the model doesn't simply default to the majority class, we implement stratified data handling and loss-function penalization.
* **Linguistic Variance**: EDA showed that spam utilizes specific "urgency" patterns and non-standard syntax. To capture this, we use an **Embedding Layer** to map words into a multi-dimensional semantic space.
* **Structural Signatures**: Since spam messages are statistically longer, we employ **Recurrent Neural Networks (RNNs)** to maintain a "memory" of long-range dependencies across the entire sequence of the message.

---

## 🧠 Technical Workflow & Approach

To deliver a production-grade classifier, we implement the following technical pillars:

### 1. Adaptive Text Vectorization
Rather than using static word counts, we implement a dynamic `TextVectorization` layer. This layer acts as an automated preprocessor that:
* **Standardizes** raw text by removing punctuation and lowercasing.
* **Tokenizes** strings into individual word units.
* **Indexes** the tokens into a vocabulary based on frequency, ensuring the model focuses on the most informative words identified during EDA.

### 2. Label Engineering & Categorical Encoding
Neural networks require purely numerical inputs for the loss function to calculate gradients. We utilize **Label Encoding** with `tf.keras.layers.StringLookup` to map our categorical targets (`ham`, `spam`) into a binary vector space ($0$ and $1$). This transformation is critical as it allows our final output neuron—equipped with a **Sigmoid** activation—to produce a probability score between $0.0$ and $1.0$.

### 3. Sequence Modeling with Bidirectional LSTMs
Short messages often hide their intent in the relationship between words. A standard LSTM reads text from left to right, but a **Bidirectional LSTM** processes the sequence in both directions simultaneously. This provides the model with a "global" context of the SMS, allowing it to understand that a word at the end of a sentence might fundamentally change the meaning of a word at the beginning.



### 4. Bayesian Hyperparameter Tuning (Optuna)
Finding the right number of neurons, embedding dimensions, and dropout rates is traditionally a manual "guess-and-check" process. We utilize **Optuna**, a state-of-the-art optimization framework, to perform **Bayesian Optimization**. By using the **Tree-structured Parzen Estimator (TPE)**, Optuna learns from each trial to narrow down the search space and find the mathematical global optimum for our validation accuracy.

### 5. High-Performance Input Pipelines (tf.data)
To ensure computational efficiency, especially when using GPU acceleration, we utilize the `tf.data` API. This allows us to implement **Prefetching** and **Caching**, which decouples data augmentation and loading from the model execution, ensuring the "training engine" never waits for the "data delivery."

---

## ⚖️ A Strategic Note on Optuna Pruning
In this notebook, you will see the inclusion of the `TFKerasPruningCallback`. **Pruning** is a mechanism that tracks the performance of an optimization trial in real-time. If a trial's performance trajectory suggests it will not surpass the current best model, Optuna terminates the trial early to save time and energy.

**Decision: We have opted NOT to activate the pruning logic during this study.**

* **The Rationale**: The SMS Spam dataset is compact, and our Bi-LSTM architecture is highly optimized for speed. On current hardware, a single trial completes in seconds.
* **The Advantage**: By allowing every trial to finish, we gather a more complete "map" of the hyperparameter landscape without the risk of early termination for a model that might have converged later in training.
* **The Purpose**: We have kept the pruning infrastructure in the code as a **professional template**. It serves to demonstrate how this workflow would be scaled for massive datasets (like the IMDB 50k or Twitter sentiment corpora) where training takes hours and pruning becomes a financial and temporal necessity.

---

## 🛠️ Implementation Roadmap
The notebook is structured into logical sections for clarity and reproducibility:
1. **Environment & Reproducibility**: Setting global seeds and initializing hardware strategy.
2. **Data Transformation**: Label encoding and stratified splitting.
3. **Pipeline Engineering**: Constructing the `tf.data` and `Vectorization` layers.
4. **The Optimization Study**: Executing the Optuna Bayesian search.
5. **Final Model Synthesis**: Training the "Winner" and performing a deep-dive into Precision, Recall, and the Confusion Matrix.

In [3]:
!pip install optuna.integration

## 1. Environment Configuration & Reproducibility
Before the architecture is initialized, we must establish a stable, deterministic environment. In deep learning, small variances in initialization can lead to different results; our goal here is consistency.

### 1.1 Library Integration & Dependency Management
We integrate **TensorFlow/Keras** for building the computational graph, **Optuna** for the optimization logic, and **Scikit-Learn** for specialized utilities. 
* `numpy`, `pandas`, `os`, `math`, `random` : some initial libaries for loading data, manipulate and mathematical and also systemical operations.
* `tensorflow`: Core of Neural Networks training, Handles the tensor operations and GPU acceleration.
* `optuna`: Manages the Bayesian search study.
* `sklearn.utils.class_weight`: Necessary for calculating the mathematical bias needed to handle our imbalanced dataset.

In [5]:
# Import neccesary libraries
import math
import numpy as np
import optuna
from optuna.integration import TFKerasPruningCallback
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.callbacks import (EarlyStopping, 
                                       ModelCheckpoint, 
                                       ReduceLROnPlateau, 
                                       TensorBoard)
import tensorflow.keras.layers as tfl
from tensorflow.keras import metrics

In [6]:
print("Available devices: \n")
# List and print all logical devices configured for TensorFlow
for device in tf.config.list_logical_devices():
    print(device.name, device.device_type)

Available devices: 

/device:CPU:0 CPU
/device:GPU:0 GPU


I0000 00:00:1765297659.784857      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


---
### 1.2 `get_strategy` Function and Distribution Strategy Activation
This function automatically detects the best available hardware accelerator and configures the corresponding TensorFlow Distribution Strategy for parallel computation. 

* **TPU Priority:** It first attempts to initialize and connect to a TPU using `TPUClusterResolver` and `tf.distribute.TPUStrategy`.
* **GPU Fallback:** If a TPU is not found, it checks for available GPUs and uses `tf.distribute.MirroredStrategy`, which is optimal for multi-GPU training.
* **CPU Default:** If neither TPU nor GPU is available, it defaults to the standard strategy.
* **Activation:** The function is called, and the resulting `strategy` object is stored. The number of active replicas (cores/GPUs) is printed, confirming the multi-device setup for training.

In [7]:
def get_strategy():
    """
    Detects and returns the best TensorFlow distribution strategy.
    - TPUStrategy for TPU(s)
    - MirroredStrategy for GPU(s)
    - Default strategy for CPU
    """
    try:
        # Try TPU first: Initialize and connect to the TPU cluster
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        print("Using TPU strategy:", type(strategy).__name__)
    except Exception:
        # If TPU not available, try GPU
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            # Use MirroredStrategy for distributed training across multiple GPUs
            strategy = tf.distribute.MirroredStrategy()
            print("Using GPU strategy:", type(strategy).__name__)
        else:
            # Fallback to CPU/default strategy
            strategy = tf.distribute.get_strategy()
            print("No TPU/GPU found. Using CPU strategy:", type(strategy).__name__)

    # Report the number of replicas, which equals the number of devices used for training
    print("REPLICAS:", strategy.num_replicas_in_sync)
    return strategy

# Execute the function to set the global distribution strategy
strategy = get_strategy()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Using GPU strategy: MirroredStrategy
REPLICAS: 1


In [8]:
# Print the confirmed number of synchronous replicas (devices) being utilized
print("REPLICAS:", strategy.num_replicas_in_sync)
# Print the TensorFlow version for reproducibility documentation
print("TensorFlow version:", tf.__version__)

REPLICAS: 1
TensorFlow version: 2.18.0


---
### 1.3 Deterministic Seeding Strategy
Neural networks initialize their weights using random distributions. To ensure that our experiments are reproducible, we set a global `SEED` value. This seed is applied to:
1.  **Python's `random` module**: For general shuffling.
2.  **Numpy**: For matrix initialization.
3.  **TensorFlow**: To ensure the graph operations and weight initializations remain identical across multiple runs on the same hardware.

In [9]:
SEED = 28
def seed_everything(SEED):
    """
    Sets the global random seeds for reproducibility across TensorFlow, NumPy, and Python's random module.
    
    Args:
        SEED (int): The integer seed value to be used.
    """
    # Set seed for the standard 'random' library
    random.seed(SEED)
    # Set seed for TensorFlow's global random operations
    tf.random.set_seed(SEED)
    # Set seed for NumPy's random operations
    np.random.seed(SEED)
    print('For reproducing purposes, everything seeded !')

# Execute the seeding function with the defined constant
seed_everything(SEED)

For reproducing purposes, everything seeded !


---
## 2. Environment Initialization & Data Ingestion

In this phase, we establish the global parameters that will govern the model's training behavior and performance efficiency. This section serves as the "Control Center" for the notebook, ensuring that all subsequent data pipelines and neural layers are aligned with the hardware's capabilities and the dataset's specific constraints.

---

### 2.1 Global Constants & Distributed Hardware Constraints
To ensure high-performance training, we define a set of hyper-constants that dictate the flow of tensors through the network. A critical component here is the calculation of the **Global Batch Size**.

* **Batch Strategy**: We use `BATCH_SIZE_PER_REPLICA = 128`. However, since we are utilizing a distributed `strategy` (e.g., GPU or TPU), we scale the global batch size by the number of active replicas:
    $$\text{Global Batch Size} = \text{Batch Size per Replica} \times N_{\text{replicas}}$$
    This ensures that the hardware is saturated effectively, maximizing throughput during the backpropagation step.
* **Sequential Limits**: We set `MAX_LENGTH = 100`, which was determined during EDA to cover the vast majority of SMS message lengths without introducing excessive padding overhead.
* **Memory Optimization**: Using `tf.data.AUTOTUNE` (assigned to `AUTO`), we delegate the tuning of CPU/GPU buffer sizes to the TensorFlow runtime, allowing for dynamic adjustment of prefetching limits based on system load.

In [10]:
# Some important hyperparameters
AUTO = tf.data.AUTOTUNE
MAX_LENGTH = 100
SHUFFLE_BUFFER_SIZE = 1024
TEST_SPLIT_RATIO = 0.2
VOCAB_SIZE = 10000
BATCH_SIZE_PER_REPLICA = 128
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync 
print(f'Global Batch size: {BATCH_SIZE}')

Global Batch size: 128


---

### 2.2 Dataset Acquisition & Path Configuration
The **SMS Spam Collection** is a specialized corpus that contains unique characters and slang, requiring specific attention during the ingestion phase.

* **File I/O**: We define the `DATA_DIR` path to point to the local Kaggle environment.
* **Encoding Rationale**: We load the CSV using `encoding='latin1'`. This is a deliberate choice because the raw SMS text frequently includes characters and symbols (common in spam advertisements) that fall outside the standard UTF-8 range. Using `latin1` ensures that no data loss occurs during the initial load.

Once the raw data is in memory, we perform structural refinement to prepare it for the preprocessing pipeline.

* **Dimensionality Reduction**: We prune the dataframe to only include the label ($y$) and the raw text ($X$), discarding redundant metadata columns.
* **Semantic Renaming**: Columns are renamed to `label` and `text`. This standardizes the schema, making the code more readable and easier to maintain. 
* **Verification**: A preliminary `.head()` check is performed. While the labels are currently categorical strings ('ham', 'spam'), they are positioned for our upcoming **Label Encoding** step, where we will transform them into the binary space $\mathcal{Y} \in \{0, 1\}$ required for numerical optimization.

In [11]:
# Path to the dataset file
DATA_DIR = '/kaggle/input/sms-spam-collection-dataset/spam.csv'

# Load the dataset with pandas, selecting only the relevant columns
df = pd.read_csv(DATA_DIR, encoding='latin1')[['v1', 'v2']]

# Rename columns for clarity
df.columns = ['label', 'text']

# Preview the first 5 rows of the dataset
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


---
## 3. Preliminary Structural Inspection

Before proceeding to data transformation, it is essential to perform a sanity check on the loaded dataframe. This section verifies the dataset's dimensions and inspects the raw alignment of labels and text to ensure no data corruption occurred during the ingestion phase.

---

### 3.1 Dataset Dimensionality & Sample Extraction
We evaluate the total volume of the corpus and examine the raw content of the primary features.
* **Shape Verification**: Confirming the total count of SMS messages and feature columns.
* **Feature Inspection**: Printing the first record to observe the raw string format of the text and its corresponding categorical label.

In [10]:
print(f"Shape of the dataset: {df.shape}")
print(f"First Element:\n{df['label'].iloc[0]}\n{df['text'].iloc[0]}")

Shape of the dataset: (5572, 2)
First Element:
ham
Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


---
### 3.2 Metadata Analysis & Label Preview
A deeper look at the statistical characteristics of the early records.
* **Word Count Estimation**: Calculating the token count of the initial sample using a standard whitespace split.
* **Target Preview**: Displaying the first five labels to verify the consistency of the categorical naming convention ('ham' vs 'spam').

In [11]:
print(f"There are {len(df)} SMS-label pairs in the dataset.\n")
print(f"First sentence has {len((df['text'].iloc[0]).split())} words.\n")
print(f"The first 5 labels are {df['label'].iloc[:5].values}")

There are 5572 SMS-label pairs in the dataset.

First sentence has 20 words.

The first 5 labels are ['ham' 'ham' 'spam' 'ham' 'ham']


---
## 4. Engineering the Modeling Datasets

In this section, we transition from raw dataframes to optimized TensorFlow Dataset objects. This process involves precise data splitting, label encoding via mapping, and the calculation of epoch steps to ensure the model iterates correctly over the entire corpus.

---

### 4.1 Stratified Splitting & Label Engineering
To ensure the integrity of our evaluation, we implement a **Stratified Split** using the `train_test_split` utility. 
* **The Logic**: By setting `stratify=df['label']`, we ensure that both the training and validation sets maintain the exact same ratio of "ham" to "spam" as the original dataset. This prevents the model from being validated on a distribution that differs from what it learned.
* **Label Representation**: The labels are converted to strings during the split to maintain consistency before they are mapped into the numerical binary space required for the loss function.

In [12]:
def datasets(df):
    '''
    Splits a DataFrame and converts it into TensorFlow Dataset objects.
    
    Args:
        df: Input pandas DataFrame containing 'text' and 'label' columns.
        
    Returns:
        train_dataset, val_dataset: TensorFlow Dataset objects for training and validation.
    '''
    
    # Split data into training and validation sets
    train_df, val_df = train_test_split(
        df,
        random_state= SEED,
        test_size= TEST_SPLIT_RATIO,
        shuffle= True,
        stratify= df['label'] # Maintains class balance
    )

    # Extract values and ensure they are strings
    train_texts = train_df['text'].astype(str).values
    train_labels = train_df['label'].astype(str).values
    val_texts = val_df['text'].astype(str).values
    val_labels = val_df['label'].astype(str).values

    # Create TensorFlow slices
    train_dataset = tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
    val_dataset = tf.data.Dataset.from_tensor_slices((val_texts, val_labels))

    print('Train and val datasets created Successfully !')

    return train_dataset, val_dataset

In [13]:
train_dataset, val_dataset = datasets(df)

Train and val datasets created Successfully !


---
### 4.2 Calculating Balanced Class Weights
As identified during EDA, our dataset suffers from a significant majority class bias. To correct this, we utilize the **Balanced Heuristic** from `scikit-learn`.
* **The Formula**: The weight for class $i$ is calculated as:
  $$W_i = \frac{N_{\text{samples}}}{N_{\text{classes}} \times N_{\text{samples},i}}$$
* **Implementation**: We map `ham` to $0$ and `spam` to $1$. The resulting weights amplify the gradients for spam messages, effectively telling the optimizer that misclassifying a spam message is "more expensive" than misclassifying a ham message.

In [23]:
def weights(df):
    '''
    Calculates balanced class weights for 'ham' and 'spam' labels.

    Args:
        df: Input pandas DataFrame with labels.

    Returns:
        class_weights: Dictionary containing weight values for each class.
    '''
    # Compute weights using sklearn's balanced method
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=[0,1],
        y=df['label'].map({'ham':0, 'spam':1})
    )
    
    # Store weights in a dictionary format
    class_weights = {0: class_weights[0], 1: class_weights[1]}
    print(f"Weights: {class_weights}")
    
    return class_weights

In [24]:
class_weights = weights(df)

Weights: {0: 0.5774093264248704, 1: 3.7295850066934406}


---
### 4.3 Deterministic Step Computation
To facilitate smooth iteration during the training loop (especially when using `.repeat()` in our pipelines), we must manually calculate the number of steps required to cover one full epoch.
* **Filtering**: We filter for non-empty texts to ensure the model doesn't process null inputs.
* **Math Logic**: We compute the `steps_per_epoch` and `validation_steps` using the ceiling of the dataset length divided by our **Global Batch Size**:
  $$\text{Steps} = \lceil \frac{\text{Total Samples}}{\text{Global Batch Size}} \rceil$$
* **Result**: This ensures that even if our dataset size is not perfectly divisible by the batch size, every single message is seen by the model during a training cycle.

In [14]:
def compute_steps(train_ds, val_ds):
    '''
    Calculates the number of steps per epoch for training and validation.

    Args:
        train_ds: The training dataset.
        val_ds: The validation dataset.

    Returns:
        steps_per_epoch, validation_steps: Integers representing calculated steps.
    '''
    # Use the original arrays before vectorization / batching
    train_texts = train_dataset.map(lambda x, y: x)
    val_texts   = val_dataset.map(lambda x, y: x)
    
    # Convert to lists safely
    train_texts = list(train_texts.as_numpy_iterator())
    val_texts   = list(val_texts.as_numpy_iterator())
    
    # Count only non-empty texts
    train_texts = [t for t in train_texts if t.strip()]
    val_texts   = [t for t in val_texts if t.strip()]
    
    # Compute steps based on total samples and BATCH_SIZE
    steps_per_epoch = math.ceil(len(train_texts) / BATCH_SIZE)
    validation_steps = math.ceil(len(val_texts) / BATCH_SIZE)

    print(f"Training steps: {steps_per_epoch}\n"
          f"Validation steps: {validation_steps}")

    return steps_per_epoch, validation_steps

In [15]:
steps_per_epoch, validation_steps = compute_steps(train_dataset, val_dataset)

Training steps: 35
Validation steps: 9


---
## 5. Feature & Label Transformation Pipelines

In this section, we construct the specialized transformation layers that translate human language and categorical labels into high-dimensional numerical tensors. By integrating these processes into the model's pipeline, we ensure that the preprocessing logic remains mathematically consistent from the training phase through to final inference.

---

### 5.1 Dynamic Text Vectorization & Vocabulary Adaptation
The `TextVectorization` layer is our primary interface for Natural Language Processing. It automates the complex task of transforming raw strings into fixed-length integer sequences that the LSTM can process.

* **Standardization Protocol**: We apply the `lower_and_strip_punctuation` setting. This normalizes the input by removing case sensitivity and punctuation noise, effectively focusing the model on the core semantics identified during our EDA phase.
* **The `.adapt()` Method**: Unlike a static dictionary, our vectorizer is **adaptive**. We map the `train_dataset` to extract text only and use `.adapt()` to build a localized vocabulary based on the top `VOCAB_SIZE` (10,000) tokens found in this specific SMS corpus.
* **Sequence Alignment & Padding**: By setting `output_sequence_length=MAX_LENGTH`, the layer ensures every output tensor has a uniform shape of 100. Shorter messages are post-padded with zeros (which the model ignores), while longer ones are truncated, satisfying the static input requirements of the neural layers.

In [16]:
def text_vectorizer(train_dataset):
    '''
    Initializes and adapts a TextVectorization layer to the training data.

    Args:
        train_dataset: The dataset used to build the vocabulary.

    Returns:
        vectorizer: The adapted TextVectorization layer.
        vocab_size: The actual size of the vocabulary built.
    '''
    # Initialize the vectorizer with specific constraints
    vectorizer = tf.keras.layers.TextVectorization(
        max_tokens= VOCAB_SIZE,
        output_sequence_length= MAX_LENGTH,
        standardize= 'lower_and_strip_punctuation',
    )

    # Extract text from dataset and batch for vocabulary adaptation
    text_only_dataset = train_dataset.map(lambda text, label: text).batch(64)
    vectorizer.adapt(text_only_dataset)
    vocab_size = vectorizer.vocabulary_size()

    print(f"Vocabulary created successfully !\nVocab size: {vocab_size}")

    return vectorizer, vocab_size

### 5.2 Tensor-Based Label Encoding via StringLookup
While we previously established the concept of label mapping, we implement it here using a high-performance `StringLookup` layer. This acts as the neural equivalent of a Label Encoder, but it is built to run natively within the TensorFlow computational graph.

* **Categorical Mapping**: The layer learns the unique mapping from the string labels ("ham", "spam") to integer indices ($0$ and $1$). This is the bridge to our **Binary Cross-Entropy** loss function.
* **Global Vocabulary Synchronization**: We concatenate labels from both the training and validation sets before adapting. This ensures that the label space is exhaustive and that the mapping remains identical across all data splits, preventing any potential "label leakage" or misalignment.
* **OOV (Out-Of-Vocabulary) Management**: By setting `num_oov_indices=0`, we strictly enforce that the model only learns from the two known categories. Since our EDA confirmed there are no hidden or missing classes, this ensures the output neuron focuses purely on the binary probability of spam.

Through these two implementations, we have effectively built the "translation" module of our network. At this point, our data is no longer viewed by the system as "words" or "strings," but as structured, numerical signals. This transformation is the prerequisite for the **Embedding Layer**, where these discrete integers will eventually gain semantic meaning in a continuous vector space.

In [17]:
def label_encoder(train_dataset, val_dataset):
    '''
    Creates and adapts a StringLookup layer to encode categorical labels into integers.

    Args:
        train_dataset: The training dataset containing labels.
        val_dataset: The validation dataset containing labels.

    Returns:
        label_encoder: The adapted StringLookup layer.
    '''
    # Extract labels from both datasets and combine them for adaptation
    train_labels = train_dataset.map(lambda text, label: label)
    val_labels = val_dataset.map(lambda text, label: label)
    labels = train_labels.concatenate(val_labels)
    
    # Initialize StringLookup without out-of-vocabulary indices
    label_encoder = tf.keras.layers.StringLookup(num_oov_indices= 0)
    label_encoder.adapt(labels)
    label_size = label_encoder.get_vocabulary()

    print(f"Label Encoder created successfully !\nUnique labels: {label_size}")

    return label_encoder

In [18]:
text_vectorizer, VOCAB_SIZE = text_vectorizer(train_dataset)

Vocabulary created successfully !
Vocab size: 8439


In [19]:
label_encoder = label_encoder(train_dataset, val_dataset)

Label Encoder created successfully !
Unique labels: ['ham', 'spam']


---
## 6. Pipeline Integratio & Tensor Verification

This section consolidates our transformation layers into a finalized `tf.data` pipeline. We apply the vectorization and encoding logic to the data streams and perform a rigorous inspection of the resulting tensors to ensure they are mathematically ready for the LSTM input.

---

### 6.1 The Preprocessing & Streaming Pipeline
We implement a high-performance transformation function that maps our raw datasets into pre-processed tensor streams.
* **Vectorization Mapping**: Every (text, label) pair is passed through our `TextVectorization` and `StringLookup` layers simultaneously.
* **Integrity Filtering**: We apply a filter to ensure that only sequences matching our `MAX_LENGTH` (100) are passed to the model, eliminating any malformed inputs.
* **Optimization Buffers**: We utilize `.shuffle()` for stochastic variance, `.batch()` with `drop_remainder=True` to maintain static tensor shapes for the GPU, and `.prefetch(AUTO)` to ensure the hardware never waits for data loading.

In [20]:
def preprocessing(dataset, text_vectorizer, label_encoder):
    '''
    Applies vectorization, encoding, and optimization pipeline to the dataset.

    Args:
        dataset: The raw TensorFlow dataset.
        text_vectorizer: Adapted TextVectorization layer.
        label_encoder: Adapted StringLookup layer.

    Returns:
        dataset: A batched and prefetched dataset ready for training.
    '''
    # Transform text and labels using the provided encoders
    dataset = (dataset
               .map(lambda text, label: (text_vectorizer(text), label_encoder(label)))
               # Keep only sequences of correct length
               .filter(lambda x, y: tf.shape(x)[0] == MAX_LENGTH)
               # Shuffle and batch for training efficiency
               .shuffle(SHUFFLE_BUFFER_SIZE)
               .batch(BATCH_SIZE, drop_remainder= True)
               # Optimize throughput with prefetching
               .prefetch(AUTO)
              )
    print('Dataset preporcessed ! READAY FOR TRAIN !')

    return dataset

In [21]:
train_dataset_pre = preprocessing(train_dataset, text_vectorizer, label_encoder)
val_dataset_pre = preprocessing(val_dataset, text_vectorizer, label_encoder)

Dataset preporcessed ! READAY FOR TRAIN !
Dataset preporcessed ! READAY FOR TRAIN !


---

### 6.2 Numerical Tensor Inspection
To verify the pipeline's output, we extract a sample batch and inspect its numerical representation.
* **Value Verification**: We print the raw integers to ensure the vectorizer is correctly mapping tokens to the vocabulary.
* **Shape Consistency**: We confirm that the output shape matches our Global Batch Size and Sequence Length ($Batch, 100$), which is the specific requirement for the model's Embedding layer.

In [21]:
# Iterate through one batch of the preprocessed dataset
for text, label in train_dataset_pre.take(1):
    # Display the numerical representation of the text
    print("Vectorized text:\n", text.numpy())
    
    # Verify the sequence length and batch size
    print("Shape:", text.shape)
    
    # Check the encoded integer value of the labels
    print("Label:", label.numpy())
    print("---")

Vectorized text:
 [[   3  194   12 ...    0    0    0]
 [ 129   41   73 ...    0    0    0]
 [2198   59  192 ...    0    0    0]
 ...
 [  23 6534   25 ...    0    0    0]
 [  41  385    9 ...    0    0    0]
 [  56   42    5 ...    0    0    0]]
Shape: (128, 100)
Label: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0]
---


---
### 6.3 Vocabulary Decoding & Logic Check
As a final sanity check, we reverse the transformation process. By mapping the integer indices back to the vocabulary tokens using `decode_sequence`, we can read the "model's view" of the SMS.
* **Integrity Check**: This confirms that our preprocessing Standardization (lowercasing and stripping punctuation) hasn't removed critical semantic meaning.
* **Label Alignment**: We verify that the decoded text still aligns logically with its binary label, ensuring the data shuffle hasn't decoupled features from their targets.

In [22]:
# Retrieve the vocabulary list from the vectorizer
vocab = text_vectorizer.get_vocabulary()

def decode_sequence(seq):
    '''
    Converts a sequence of token indices back into a human-readable string.

    Args:
        seq: A tensor containing token indices.

    Returns:
        A string of decoded tokens joined by spaces, excluding padding (0).
    '''
    return " ".join([vocab[i] for i in seq.numpy() if i != 0])

# Iterate through three batches to verify decoding and labels
for text, label in train_dataset_pre.take(3):
    # Decode the first sample in the batch
    print("Decoded text:", decode_sequence(text[0]))
    # Print the corresponding label for that sample
    print("Label:", label[0].numpy())
    print("---")

Decoded text: in which place do you want da
Label: 0
---
Decoded text: hey so this sat are we going for the intro pilates only or the kickboxing too
Label: 0
---
Decoded text: ummmmmaah many many happy returns of d day my dear sweet heart happy birthday dear
Label: 0
---


---
## 7. Neural Architecture Design & Bayesian Optimization

In this phase, we define the computational "brain" of our classifier and the automated intelligence used to tune it. We transition from manual configuration to **Automated Machine Learning (AutoML)** by utilizing **Optuna** to navigate the complex, high-dimensional space of hyperparameters, ensuring our Bi-LSTM reaches its maximum predictive potential.

---

### 7.1 The Bidirectional LSTM Architecture
Our model is specifically engineered to handle the sequential and often deceptive nature of SMS text. Unlike standard feed-forward networks, this architecture maintains a temporal memory of the message content.

* **Embedding Layer**: This acts as a dynamic lookup table that maps discrete integer indices into a continuous vector space. It learns the semantic proximity of tokens, effectively grouping words like "win," "prize," and "claim" into a similar mathematical "meaning-space."
* **Bidirectional LSTM Core**: Standard LSTMs process text unidirectionally (left-to-right). Our **Bidirectional** wrapper duplicates the LSTM layer—one processes the SMS from start to finish, and the other from finish to start. This dual-lens approach is critical for spam detection, as it captures context that might be split across the message, such as a "Greeting" at the beginning and a "Malicious Link" at the end.
* **Dynamic Dense Bottleneck**: Following the LSTM, we implement a modular series of **Dense** layers with **ReLU** activations. These layers perform high-level feature extraction, condensing the sequential patterns identified by the LSTM into a final classification signal.
* **Regularization (Dropout)**: To prevent the model from memorizing specific training samples (overfitting), we apply Dropout. This randomly deactivates a percentage of neurons during each training step, forcing the network to learn more robust and generalized features.

In [22]:
def create_model(hparams):
    '''
    Builds a dynamic Bidirectional LSTM model based on provided hyperparameters for using Optuna.

    Args:
        hparams: Dictionary containing tuning parameters like embedding_dim, 
                 lstm_units, dropout_rate, and dense layer configurations.

    Returns:
        model: A uncompiled Keras Model instance.
    '''
    # Define the input layer with the fixed maximum sequence length
    inputs = tf.keras.Input(shape= (MAX_LENGTH, ))
    
    # Embedding layer to convert token indices into dense vectors
    x = tfl.Embedding(VOCAB_SIZE, hparams["embedding_dim"])(inputs)
    
    # Bidirectional LSTM to capture context from both directions
    x = tfl.Bidirectional(tfl.LSTM(hparams["lstm_units"]))(x)
    
    # Dropout layer to prevent overfitting
    x = tfl.Dropout(hparams["dropout_rate"])(x)
    
    # Dynamically add the specified number of dense layers
    num_dense_layers = hparams["num_dense_layers"]
    for i in range(num_dense_layers):
        units = hparams[f"dense_layers_{i}"]
        x = tfl.Dense(units, activation= 'relu')(x)
        
    # Output layer with sigmoid activation for binary classification
    outputs = tfl.Dense(1, activation= 'sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)

    model.summary()
    return model

### 7.2 The Optuna Optimization Paradigm: Beyond Random Search
Hyperparameter optimization is the process of finding the configuration that maximizes our objective function (Validation Accuracy). In deep learning, this is a "Black-Box" problem because we cannot calculate a direct gradient for parameters like "number of layers" or "learning rate." 

#### 1. Bayesian Optimization & TPE
Optuna utilizes the **Tree-structured Parzen Estimator (TPE)**. Unlike Grid Search, which is slow and exhaustive, or Random Search, which ignores previous history, TPE is **Bayesian**. 
* **The "History" Effect**: TPE builds a probability model (a surrogate model) of the objective function based on the results of past trials. 
* **Exploration vs. Exploitation**: It uses this model to decide where to "sample" next, intelligently balancing the search between unknown regions and regions that have historically yielded high scores.

#### 2. Defining the Search Space (Suggest API)
Optuna allows us to define a dynamic, conditional search space. This makes the architecture itself flexible:
* **Categorical Suggestions**: Used for fixed choices, such as selecting an embedding dimension (16, 32, or 64).
* **Float Suggestions (Log-Uniform)**: Crucial for the `learning_rate` and `weight_decay`. Because these values operate across different orders of magnitude ($10^{-5}$ vs $10^{-3}$), searching on a logarithmic scale ensures the optimizer samples small values just as effectively as large ones.
* **Integer Suggestions (Step-based)**: Used for the number of Dense layers or units, allowing the search to move in discrete, logical increments.

#### 3. The Lifecycle of an Optuna Trial
Each trial follows a standardized, automated workflow:
1. **Suggestion**: Optuna’s TPE sampler provides a unique hyperparameter set.
2. **Construction**: The `create_model` function builds a fresh Bi-LSTM graph based on those suggestions.
3. **Execution**: The model is compiled within the distributed `strategy.scope()` and trained.
4. **Observation**: The validation accuracy is reported back to the `study` object to inform future trials.

In [75]:
def objective(trial, epochs= 10):
    '''
    Optuna objective function for hyperparameter optimization.
    
    Args:
        trial: A trial object from Optuna.
        epochs: Number of training epochs per trial.
        
    Returns:
        val_acc: The maximum validation accuracy achieved during the trial.
    '''
    # Define the search space for architectural hyperparameters
    hparams = {
        "embedding_dim" : trial.suggest_categorical("embedding_dim", [16, 32, 64]),
        "lstm_units": trial.suggest_categorical("lstm_units", [16, 32, 64]),
        "dropout_rate": trial.suggest_float("dropout_rate", 0.1, 0.5),
        "num_dense_layers": trial.suggest_int("num_dense_layers", 1, 3)
    }

    # Define optimization-related hyperparameters
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)

    # Dynamically suggest units for each dense layer based on the number of layers
    for i in range(hparams["num_dense_layers"]):
        hparams[f"dense_layers_{i}"] = trial.suggest_int(
            f"dense_layers_{i}", 16, 128, step= 16
        )

    # Initialize model and optimizer within the distributed strategy scope
    with strategy.scope():
        
        model = create_model(hparams)
        
        optimizer = tf.keras.optimizers.AdamW(
            learning_rate= learning_rate,
            weight_decay= weight_decay,
            beta_1= 0.9,
            beta_2= 0.999,
            epsilon= 1e-7
        )
        loss = tf.keras.losses.BinaryCrossentropy(label_smoothing= 0)
        
        metrics_list = [
            metrics.BinaryAccuracy(name= 'accuracy'),
            metrics.Recall(name= 'recall'),
            metrics.Precision(name= 'precision'),
            metrics.AUC(name= 'auc')
        ]
        
        model.compile(
            optimizer= optimizer,
            loss= loss,
            metrics= metrics_list
        )

    # Set up dynamic learning rate reduction on plateau
    reduce_lr = ReduceLROnPlateau(
        monitor= 'val_loss',
        factor= 0.3,
        patience= 2,
        min_lr= 1e-6
    )
    
    callbacks = [reduce_lr]

    # Train the model using preprocessed datasets and class weights
    history = model.fit( 
        train_dataset_pre.repeat(), 
        validation_data= val_dataset_pre.repeat(), 
        epochs= epochs, 
        steps_per_epoch= steps_per_epoch, 
        validation_steps= validation_steps, 
        callbacks= callbacks, 
        class_weight= class_weights 
    )
        
    # Track the best validation accuracy for Optuna to maximize
    val_acc = max(history.history["val_accuracy"])
    
    return val_acc

### 7.3 Study Execution & Strategy
We initiate the optimization `study` with a clear goal: **Maximize the Validation Accuracy over 30 trials.**

* **Distributed Strategy**: The model is built within the `strategy.scope()`. This ensures that the trials are hardware-optimized, utilizing available GPUs or TPUs to reduce the search time.
* **AdamW Optimizer**: We use the Adam optimizer with Weight Decay (AdamW). This decouples weight decay from the gradient updates, providing better regularization and helping the model generalize to unseen SMS patterns.
* **Multi-Metric Monitoring**: While Optuna optimizes for accuracy, we track **Precision**, **Recall**, and **AUC** in parallel. This ensures that we are not just getting a high "score," but specifically building a model that minimizes the "False Positives" (Legitimate mail marked as spam) which were identified as a critical concern in our EDA.

In [76]:
# Create an Optuna study object to find the best hyperparameter configuration
study = optuna.create_study(direction= 'maximize')
# Start the optimization process over a fixed number of trials
study.optimize(lambda trial: objective(trial, epochs= 10), n_trials= 30)

[I 2025-12-09 17:31:03,635] A new study created in memory with name: no-name-ec9dc6e6-33de-4716-b738-08f978776c7e
/tmp/ipykernel_47/930312181.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
/tmp/ipykernel_47/930312181.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)


Model: "functional_69"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_69 (InputLayer)     │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_69 (Embedding)        │ (None, 100, 16)        │       135,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_69                │ (None, 32)             │         4,224 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_69 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_201 (Dense)               │ (None, 96)             │         3,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_202 (Dense)               │ (None, 1)              │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,513 (556.69 KB)

 Trainable params: 142,513 (556.69 KB)

...

 Total params: 143,057 (558.82 KB)

 Trainable params: 143,057 (558.82 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.8035 - auc: 0.5402 - loss: 0.7004 - precision: 0.1374 - recall: 0.0842 - val_accuracy: 0.8594 - val_auc: 0.6819 - val_loss: 0.6865 - val_precision: 0.1667 - val_recall: 0.0130 - learning_rate: 6.7344e-05
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.8223 - auc: 0.6656 - loss: 0.6739 - precision: 0.2468 - recall: 0.1984 - val_accuracy: 0.8698 - val_auc: 0.8328 - val_loss: 0.6846 - val_precision: 0.5299 - val_recall: 0.3949 - learning_rate: 6.7344e-05
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.8277 - auc: 0.7696 - loss: 0.6702 - precision: 0.3646 - recall: 0.5008 - val_accuracy: 0.8750 - val_auc: 0.8949 - val_loss: 0.6793 - val_precision: 0.5214 - val_recall: 0.7922 - learning_rate: 6.7344e-05
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8583 - auc: 0.8700 - loss: 0.6949 - precision: 0.4944 - recall: 0.6822 - val_accuracy: 0.8802 - val_auc: 0.9259 - val_loss: 0.66

[I 2025-12-09 17:40:16,713] Trial 29 finished with value: 0.9053819179534912 and parameters: {'embedding_dim': 16, 'lstm_units': 16, 'dropout_rate': 0.4386399914823313, 'num_dense_layers': 1, 'learning_rate': 6.734387891109436e-05, 'weight_decay': 6.313791697836341e-05, 'dense_layers_0': 112}. Best is trial 4 with value: 0.9887152910232544.


---
### 7.4 Extraction of the "Champion" Hyperparameters
Upon the completion of the 30-trial Bayesian search, we perform a "Result Harvesting" step. 

* **State Retrieval**: We programmatically identify the `best_trial` from the study object, which contains the specific combination of neurons, learning rates, and embedding sizes that achieved the highest validation performance.
* **Configuration Snapshot**: These parameters are stored in a static `best_hparams` dictionary. By capturing this "Champion" configuration, we ensure that our final training run is perfectly reproducible and utilizes the most mathematically efficient architecture discovered during the optimization lifecycle.

In [77]:
# print best trial hyperparameters for training model in final with these hypers
print("Best trial:")
trial = study.best_trial
print(trial.params)

Best trial:
{'embedding_dim': 16, 'lstm_units': 16, 'dropout_rate': 0.4442144855683424, 'num_dense_layers': 1, 'learning_rate': 0.0003230576384522256, 'weight_decay': 0.0006193087602825927, 'dense_layers_0': 128}


In [80]:
# Best hyperparameters from Optuna
best_hparams = {
    'embedding_dim': 16, 
    'lstm_units': 16, 
    'dropout_rate': 0.4442144855683424, 
    'num_dense_layers': 1, 
    'learning_rate': 0.0003230576384522256, 
    'weight_decay': 0.0006193087602825927, 
    'dense_layers_0': 128
}

## 8. Final Model Synthesis & Robust Training Strategy

In this concluding phase, we consolidate our previous work by synthesizing a "Champion" model using the optimal hyperparameters discovered by Optuna. We implement a rigorous training regimen designed to maximize generalization and prevent overfitting through the use of advanced Keras callbacks.

---

### 8.1 Architecting the Optimized "Champion" Model
We define the final model structure using a specialized constructor function. This ensures that the architecture is perfectly aligned with the parameters found during our 30-trial search.

* **Hardwired Best Params**: The function dynamically builds the **Embedding** and **Bi-LSTM** layers using the optimized unit counts.
* **Conditional Dense Logic**: We implement a loop to generate the exact number of hidden dense layers (`num_dense_layers`) that Optuna determined was most effective for extracting high-level features from the SMS sequences.
* **Probabilistic Output**: The final layer remains a single neuron with **Sigmoid** activation, providing the decimal probability of a message being spam.

In [81]:
def create_final_model(hparams):
    '''
    Constructs the final model architecture using the optimized hyperparameters.

    Args:
        hparams: Dictionary of best hyperparameters found during the study.

    Returns:
        model: A Keras Model instance ready for final training or inference.
    '''
    # Define input shape based on pre-defined MAX_LENGTH
    inputs = tf.keras.Input(shape=(MAX_LENGTH,))
    
    # Embedding layer to process tokenized text
    x = tf.keras.layers.Embedding(VOCAB_SIZE, hparams["embedding_dim"])(inputs)
    
    # Bidirectional LSTM for temporal feature extraction
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hparams["lstm_units"]))(x)
    
    # Dropout for regularization
    x = tf.keras.layers.Dropout(hparams["dropout_rate"])(x)

    # Reconstruct the optimized dense layer structure
    for i in range(hparams["num_dense_layers"]):
        units = hparams[f"dense_layers_{i}"]
        x = tf.keras.layers.Dense(units, activation='relu')(x)

    # Final sigmoid output for binary classification
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    
    model.summary()
    return model

### 8.2 Distributed Compilation & Optimization Strategy
Once the model is instantiated, we must configure its learning parameters within the hardware-aware `strategy.scope()`.

* **AdamW Integration**: We use the **AdamW** optimizer, which decouples weight decay from gradient updates. This allows us to apply the exact `weight_decay` found by Optuna to penalize complex weights, significantly improving the model's ability to generalize to new, unseen text.
* **Loss Function**: We utilize **Binary Cross-Entropy**, the mathematical standard for binary classification, which measures the "distance" between the model's predicted probability and the true label (0 or 1).
* **Multi-Metric Evaluation**: We compile the model with a comprehensive suite of metrics—**Accuracy**, **Precision**, **Recall**, and **AUC**. This provides a holistic view of performance, ensuring the model is balanced in its ability to catch spam without flagging legitimate "Ham" messages.

In [82]:
# Initialize and compile the final model using the best hyperparameters found
with strategy.scope():
    # Instantiate the final model architecture
    final_model = create_final_model(best_hparams)

    # Configure the AdamW optimizer with tuned learning rate and weight decay
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=best_hparams["learning_rate"],
        weight_decay=best_hparams["weight_decay"]
    )

    # Set up binary crossentropy loss and target metrics
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing= 0)
    metrics_list = ['accuracy', 'precision', 'recall', 'auc']
    
    # Compile the model within the distribution strategy scope
    final_model.compile(
        optimizer= optimizer,
        loss= loss,
        metrics= metrics_list
    )

Model: "functional_99"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_99 (InputLayer)     │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_99 (Embedding)        │ (None, 100, 16)        │       135,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_99                │ (None, 32)             │         4,224 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_99 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_288 (Dense)               │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_289 (Dense)               │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 143,601 (560.94 KB)

 Trainable params: 143,601 (560.94 KB)

 Non-trainable params: 0 (0.00 B)

---
### 8.3 Defensive Training via Callbacks
To ensure the model reaches its peak performance without drifting into overfitting, we implement a suite of strategic callbacks:

1.  **EarlyStopping**: We monitor the `val_loss` with a patience of 10 epochs. If the model stops improving on the validation set, the training is halted immediately, and `restore_best_weights=True` ensures we keep the model at its highest performing state.
2.  **ModelCheckpoint**: This automatically serializes and saves the best iteration of our model to `SMS_Spam_best_model.keras`. This guarantees that even if a system failure occurs, we have preserved the most accurate version of our classifier.
3.  **ReduceLROnPlateau**: When the model's progress on validation loss plateaus, this callback dynamically reduces the learning rate by a factor of 0.3. This allows the optimizer to "fine-tune" the weights in the local minima, achieving higher precision.
4.  **TensorBoard**: We enable deep telemetry by logging histograms and training curves to `/kaggle/working/logs`. This allows for a post-training visual audit of weight distributions and gradient stability.

In [79]:
# Monitor validation loss to stop training when performance plateaus
early_stop = EarlyStopping(
    monitor= 'val_loss',
    patience= 10,
    mode= 'min',
    restore_best_weights= True
)

# Save the model only when validation loss improves
checkpoint = ModelCheckpoint(
    'SMS_Spam_best_model.keras',
    monitor='val_loss',
    mode= 'min',
    save_best_only=True,
    verbose = 1
)

# Dynamically reduce learning rate to fine-tune weights on local minima
reduce_lr = ReduceLROnPlateau(
    monitor= 'val_loss',
    factor= 0.3,
    patience= 3,
    min_lr= 1e-6
)

# Export training metrics for visualization in TensorBoard
tb_cb = TensorBoard(
    log_dir= '/kaggle/working/logs',
    histogram_freq= 1
)

# Consolidate all callbacks for the model.fit() process
callbacks = [early_stop, reduce_lr, checkpoint, tb_cb]

### 8.4 Executing the Final Convergence
We initiate the final training session with a maximum of **50 epochs**. 

* **Class Weight Application**: We continue to apply the calculated `class_weights` to ensure the model remains vigilant for the minority "Spam" class during this final refinement.
* **Repetitive Streaming**: By using `.repeat()` on our `tf.data` pipelines alongside the calculated `steps_per_epoch`, we ensure a continuous, high-throughput flow of data that utilizes every single sample in our training and validation sets.

Through this process, we transform our initial raw SMS data into a highly specialized, context-aware neural engine capable of production-grade spam filtering.

In [83]:
# Retrain with full epochs
FINAL_EPOCHS = 50

history_final = final_model.fit(
    train_dataset_pre.repeat(),
    validation_data=val_dataset_pre.repeat(),
    epochs=FINAL_EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks,
    class_weight=class_weights
)


Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8212 - auc: 0.6318 - loss: 0.6830 - precision: 0.2506 - recall: 0.1417
Epoch 1: val_loss improved from inf to 0.64329, saving model to SMS_Spam_best_model.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.8221 - auc: 0.6347 - loss: 0.6830 - precision: 0.2553 - recall: 0.1436 - val_accuracy: 0.9123 - val_auc: 0.9172 - val_loss: 0.6433 - val_precision: 0.7027 - val_recall: 0.6460 - learning_rate: 3.2306e-04
Epoch 2/50
34/35 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8839 - auc: 0.8873 - loss: 0.6480 - precision: 0.5624 - recall: 0.6376
Epoch 2: val_loss improved from 0.64329 to 0.48330, saving model to SMS_Spam_best_model.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - accuracy: 0.8843 - auc: 0.8877 - loss: 0.6465 - precision: 0.5633 - recall: 0.6402 - val_accuracy: 0.8628 - val_auc: 0.9265 - val_loss: 0.4833 - val_precision: 0.4886 - val_recall: 0.8487 - learning_rate: 3.2306e-04
Epoch 3/50
35/35 ━━━━━━━━━━━

---
## 9. Final Conclusion & Performance Synthesis

This project successfully transitioned from raw text exploration to a high-performance **Bi-LSTM** classification engine. By leveraging **Bayesian Optimization** via Optuna and a hardware-aware **tf.data** pipeline, we have engineered a solution that is both mathematically optimized and computationally efficient.

---

### 9.1 Empirical Training Analysis
Our final training logs reveal a highly successful convergence. The model demonstrated rapid learning, as evidenced by the performance trajectory:

* **Rapid Convergence**: By Epoch 5, the model achieved a **Validation AUC of 0.9880**, indicating an exceptional ability to distinguish between the two classes with very few false positives.
* **Peak Generalization**: The "Champion" model reached its optimal state at **Epoch 11**, achieving:
    * **Validation Accuracy**: ~98.8%
    * **Validation Precision**: 97.3%
    * **Validation Recall**: 93.5%
* **Stability through Regularization**: The use of **ReduceLROnPlateau** was instrumental. As the loss plateaued around Epoch 15, the learning rate was systematically decayed (from $3.23 \times 10^{-4}$ to $8.72 \times 10^{-6}$), allowing the model to settle into a stable local minimum without oscillating.



---

### 9.2 Key Technical Takeaways
The success of this architecture can be attributed to three primary strategic decisions:

1.  **Optimized Semantic Depth**: Optuna identified that a relatively lean architecture (16 embedding dimensions and 16 LSTM units) provided the best generalization. This prevents the "memorization" of noise—a common pitfall in small text datasets.
2.  **Imbalance Mitigation**: The integration of **Class Weights** allowed the model to maintain a high **Recall (~93.5%)** despite the heavy skew toward "Ham" messages. The model "learned" that missing a spam message was significantly more costly than misclassifying a legitimate one.
3.  **Sequential Intelligence**: The **Bidirectional LSTM** proved highly effective at capturing the "urgency" and "reward" patterns typical of modern spam, processing context from both directions to ensure no deceptive signals were missed.

---

### 9.3 Production Readiness
The final model has been serialized as `SMS_Spam_best_model.keras`. With a **Validation AUC of 0.98+**, this system is ready for deployment in real-time filtering environments. The inclusion of the `TextVectorization` and `StringLookup` layers within the preprocessing pipeline ensures that the model can be served as a "black box"—accepting raw string inputs and returning highly accurate spam probability scores with minimal latency.



---
**End of Notebook**